In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import py_vollib_vectorized as vollib


TRADIER_API_KEY = "Jta3vCKbfk5HxsN8hHqEp4HlyPku"
headers = {'Authorization': "Bearer {}".format(TRADIER_API_KEY), 'Accept': 'application/json'}
BASE_URL = "https://sandbox.tradier.com"

In [39]:

def get_first_val_in_table(table):
    first_row = table.find('tr')
    if first_row:
        first_field = first_row.find('td')
        if first_field:
            enclosed = first_field.find('div')
            if enclosed:
                return enclosed.text
    return ''

def get_si_data_from_table(table,df=True):
    data = []
    cols = []
    rows = table.findAll('tr')
    for row in rows:
        col,val = row.findAll('td')
        cols.append(col.find('div').text.strip())
        data.append(val.text.strip())
    if df: return pd.DataFrame([data[2:]],columns=cols[2:]) # first 2 fields are bs
    return cols,data
        

def get_short_interest_for_security(ticker):
    response = requests.get("https://shortsqueeze.com/?symbol={}&submit=Short+Quote%E2%84%A2".format(ticker))
    soup = BeautifulSoup(response.text,'html.parser')
    si_table = [table for table in soup.findAll('table') if get_first_val_in_table(table) == 'Short Squeeze Ranking™'][0]
    return get_si_data_from_table(si_table)


def get_options_for_security(stock):
    response = requests.get('{}/v1/markets/options/lookup'.format(BASE_URL),
        params={'underlying': stock},
        headers=headers
    )

    json_response = response.json()
    return json_response['symbols'][0]['options']


def get_quotes_for_options(options,batch_size=25):
    chains = []
    for i in range(0, len(options),batch_size):
        options_batch = options[i:i+batch_size]
        response = requests.get('{}/v1/markets/quotes'.format(BASE_URL),
            params={'symbols': ','.join(options_batch), 'greeks': 'true'},
            headers=headers
        )    
        json_response = response.json()
        chains += json_response['quotes']['quote']
    return chains
    
def get_quote_for_security(stock):
    response = requests.get('{}/v1/markets/quotes'.format(BASE_URL),
        params={'symbols': stock, 'greeks': 'false'},
        headers=headers
    )    
    json_response = response.json()
    return json_response['quotes']['quote']
    

def get_most_shorted_stocks(df=True):
    table_cols = ['Symbol\nSymbol',
                 'Company Name',
                 'Price',
                 'Chg% (1D)',
                 'Chg% (YTD)',
                 'Short Interest',
                 'Short Date',
                 'Float',
                 'Float Shorted (%)']
    url = "https://www.marketwatch.com/tools/screener/short-interest"
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    si_table = soup.findAll('table')[0]
    si_cols =  [col.text.strip('\n') for col in si_table.find('thead').find('tr').findAll('th')]
    assert(si_cols == table_cols)
    rows = si_table.findAll('tr')[1:]
    data = []
    for row in rows: data.append([val.text.strip('\n') for val in row.findAll('td')])
    if df: return pd.DataFrame(data,columns=si_cols)
    return data


In [40]:
get_short_interest_for_security('WISH')

,Short Interest Ratio (Days To Cover),Short Percent of Float,Short % Increase / Decrease,Short Interest (Current Shares Short),Shares Float,Short Interest (Prior Shares Short)
0,1.3,8.76 %,-23 %,"24,120,000","275,240,000","31,430,000"
